# Language Modeling using Ngram

In this tutorial we will write a code using NLTK which is a natural language processing library for python.

In [25]:
import math
import nltk
from nltk import bigrams
from collections import Counter, defaultdict

First we import necessary library such as math, nltk, bigram, and collections. We also already download 'gutenberg' corpus from nltk to use as input data for our language modeling task.

In [4]:
nltk.corpus.gutenberg.fileids()

[u'austen-emma.txt',
 u'austen-persuasion.txt',
 u'austen-sense.txt',
 u'bible-kjv.txt',
 u'blake-poems.txt',
 u'bryant-stories.txt',
 u'burgess-busterbrown.txt',
 u'carroll-alice.txt',
 u'chesterton-ball.txt',
 u'chesterton-brown.txt',
 u'chesterton-thursday.txt',
 u'edgeworth-parents.txt',
 u'melville-moby_dick.txt',
 u'milton-paradise.txt',
 u'shakespeare-caesar.txt',
 u'shakespeare-hamlet.txt',
 u'shakespeare-macbeth.txt',
 u'whitman-leaves.txt']

Project gutenberg is a public collaboration to provide free e-book to the internet. 
The list of project gutenberg's available books in nltk is as listed above.

In [8]:
print 'total sentences in Moby Dick :\t'+ str(len(nltk.corpus.gutenberg.sents('melville-moby_dick.txt')))
print 'total word counts in Moby Dick :\t'+ str(len(nltk.corpus.gutenberg.words('melville-moby_dick.txt')))
print 'total vocabuary in Moby Dick :\t'+ str(len(set(nltk.corpus.gutenberg.words('melville-moby_dick.txt'))))

total sentences in Moby Dick :	10059
total word counts in Moby Dick :	260819
total vocabuary in Moby Dick :	19317


In [26]:
sentences = nltk.corpus.gutenberg.sents('melville-moby_dick.txt')
print 'sentence example:'
print sentences[5]

sentence example:
[u'He', u'loved', u'to', u'dust', u'his', u'old', u'grammars', u';', u'it', u'somehow', u'mildly', u'reminded', u'him', u'of', u'his', u'mortality', u'.']


In [27]:
train = sentences[:9000]
test = sentences[9000:]

We separate out input into 2 sets, input and test data. Input data is consisted of the first 9000 sentences in the book. Test data is consisted of 1059 last sentences.

In [69]:
model = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in train:
    for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True): #None I go to school . None
        model[w1][w2] +=1

Next, we created our simple language model using only bigrams. The left and right padding are for the before and after marker which use None as a token. We counted the occurrences of a couple of words in the book.

In [70]:
for w1 in model:
    unigram = float(sum(model[w1].values()))
    for w2 in model[w1]:
        #model[w1][w2] = model[w1][w2]/total_count
        model[w1][w2] = math.log(model[w1][w2]/unigram)

The bigram problability is calculated by this formula log(P(w2,w1)) = log(Count(w2,w1)/Count(w1))

In [19]:
print 'log(P("man", "old")) :'+ str(model['old']['man'])

log(P("man", "old")) :-1.76314893519


In [30]:
def calculate_sentence_ln_prob(sentence):
    if type(sentence) is str:
        sent =sentence.split(' ')
    else:
        sent = sentence
    prob =.0
    for i in range(0,len(sent)-1):
        if i==0:
            prob+=model[None][sent[i]]
        elif i==len(sent)-2:
            prob+=model[sent[i+1]][None]
        prob +=model[sent[i]][sent[i+1]]
    return prob

In [186]:
print calculate_sentence_prob('The old man is swimming in the sea .')
print calculate_sentence_prob('The old man and swimming in the sea .')
#print model[None]['The']+model['The']['old']+model['old']['man']+model['man'][None]

-24.1992792398
-33.9052909877


This method is for calculating the problability of a sentence. However the sentence usually lack the start and stop marker so we need to add it ourself.

In [73]:
def perplexity(test):
    prob=.0
    words=0
    for sent in test:
        words+=len(sent)
        prob += calculate_sentence_prob(sent)
    #return math.pow(prob,-words)
    return math.exp(-prob/words)

In [74]:
print perplexity(test)

12.5136522426


This method is for perplexity calculation, which after taking logarithm of e, can be expressed as 

PERPLEXITY = e^((-1/word_count)*(Sum of all sentences problability))

In [75]:
#Laplace Smoothing
model = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in train:
    for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
        model[w1][w2] +=1
v = len(set(nltk.corpus.gutenberg.words('melville-moby_dick.txt')))
for w1 in model:
    unigram_count = float(sum(model[w1].values()))
    for w2 in model[w1]:
        model[w1][w2] = math.log((model[w1][w2]+1)/(unigram_count+v))
        #model[w1][w2]/=total_count
print perplexity(test)

101.708519808


In [77]:
#interpolation
model = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in train:
    for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
        model[w1][w2] +=1
total = len(nltk.corpus.gutenberg.words('melville-moby_dick.txt'))
for w1 in model:
    unigram_count = float(sum(model[w1].values()))
    for w2 in model[w1]:
        model[w1][w2] = math.log(((0.7*model[w1][w2])/(unigram_count))+(0.3*unigram_count/total))
        #model[w1][w2]/=total_count
print perplexity(test)

9.9617761056


Bigram interpolation is consisted of P(w2,w1) + P(w1)

In [90]:
import random
 
text = [None]
sentence_finished = False
 
while not sentence_finished:
    r = random.random()
    accumulator = .0
 
    for word in model[text[-1:][0]].keys():
        accumulator += math.exp(model[text[-1:][0]][word])
        if accumulator >= r:
            text.append(word)
            break
 
    if text[-1:] == [None]:
        sentence_finished = True
print ' '.join([t for t in text if t])

Cruppered with mild looking sky .
